# ISA

This notebook provides wrapper functions for calling the [ISA (Iterative Subtractive Alignment) algorithm](https://archives.ismir.net/ismir2021/paper/000101.pdf).  Running this algorithm requires installing some other software, which is described below.  This notebook implements the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.

## Offline Processing

In the offline processing stage, three things are computed and stored in the `cache/` folder:
- CQT features for the orchestra recording
- CQT features for the full mix recording

In [ ]:
import numpy as np
import pandas as pd
import import_ipynb
import System_OfflineDTW
import system_utils
import align_tools
import sonify_tools
import os
import os.path
import subprocess
import librosa as lb
from shutil import which
from hmc_mir.align import isa, dtw
import matplotlib.pyplot as plt
from numba import jit, njit, prange
import time

In [ ]:
def split_into_five_second_segments(piano_cqt):
    n = piano_cqt.shape[1]
    return [[i, min(i+215, n)] for i in range(0, n, 215)]

In [ ]:
def offline_processing(scenario_dir, cache_dir, hop_length, alg='cqt'):
    '''
    Carries out the same offline processing steps as the simple offline DTW system.
    
    Args:
        scenario_dir: The scenario directory to process
        cache_dir: The location of the cache directory
        hop_length: The hop length in samples used when computing chroma features
        alg: The chroma feature algorithm to use. Must be one of 'cqt', 'bcqt', or 'chroma'.
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''
    # setup
    system_utils.verify_scenario_dir(scenario_dir)
    if os.path.exists(cache_dir):
        # print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # setup
        os.makedirs(cache_dir)

        o_file = f'{scenario_dir}/o.wav'
        y_o, sr = lb.core.load(o_file)
        F_o = isa.calculate_cqt(y_o, sr, hop_length)

        po_file = f'{scenario_dir}/po.wav'
        y_po, sr = lb.core.load(po_file)
        F_po = isa.calculate_cqt(y_po, sr, hop_length)

        np.save(f'{cache_dir}/o_cqt.npy', F_o)
        np.save(f'{cache_dir}/po_cqt.npy', F_po)

    return

In [ ]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/o_cqt.npy'), f'o_cqt.npy missing from {indir}'
    assert os.path.exists(f'{indir}/po_cqt.npy'), f'po_cqt.npy missing from {indir}'

## Online Processing

In the online processing stage, the following steps are done:
- estimate the P-PO alignment using standard subsequence DTW with chroma features
- perform spectral subtraction of P from PO, which produces an estimate of the O CQT features in the PO recording
- estimate the PO_O_est - O alignment using subsequence DTW with chroma features
- infer the P-O alignment

### Wrapper Implementation

In [ ]:
def online_processing(scenario_dir, out_dir, cache_dir, hop_length, alg='cqt'):
    '''
    Carries out `online' processing using the ISA algorithm.
    
    Args:
        scenario_dir: The scenario directory to process
        out_dir: The directory to put results, intermediate files, and logging info
        cache_dir: The cache directory
        hop_length: The hop length in samples used when computing cqt features
        alg: The chroma feature algorithm to use. Must be one of 'cqt', 'bcqt', or 'chroma'.

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    timestamps = []
    
    # verify & setup
    system_utils.verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir), f'Output directory {out_dir} already exists.'
    os.makedirs(out_dir)

    # compute features
    p_file = f'{scenario_dir}/p.wav'
    y, sr = lb.core.load(p_file)
    hop_sec = hop_length / sr
    F_p = isa.calculate_cqt(y, sr, hop_length)  # piano CQT
    F_po = np.load(f'{cache_dir}/po_cqt.npy') # full mix CQT
    F_o = np.load(f'{cache_dir}/o_cqt.npy') # orchestra CQT

    # preliminary P-PO alignment (just to select the appropriate section of PO to perform ISA)
    timestamps.append(time.time())
    buffer = int(1/hop_sec) # include a buffer of 1 sec on either end of estimated alignment
    C = align_tools.cosine_dist(isa.cqt_to_chroma(F_p), isa.cqt_to_chroma(F_po))
    D, B, p_po_path = dtw.dtw(C, np.array([[1,1],[1,2],[2,1]]),[1,1,2], True)
    F_po_match = F_po[:, max(p_po_path[1,0]-buffer, 0):min(p_po_path[1,-1]+buffer, F_po.shape[1])]
    po_offset = max(p_po_path[1,0]-buffer, 0)

    timestamps.append(time.time())
    segments_p = split_into_five_second_segments(F_p)

    if alg == 'cqt':
        F_po_o_est, wp_AB = isa.isa_cqt(F_p, F_po_match, segments_p)
    elif alg == 'bcqt':
        F_po_o_est, wp_AB = isa.isa_bcqt(F_p, F_po_match, segments_p)
    elif alg == 'chroma':
        F_po_o_est, wp_AB = isa.isa_chroma(F_p, F_po_match, segments_p)
    else:
        raise ValueError(f'alg must be one of cqt, bcqt, or chroma.  Received {alg}')

    wp_AB[1,:] = wp_AB[1,:] + po_offset # account for offset in P-PO alignment

    timestamps.append(time.time())
    C = align_tools.cosine_dist(isa.cqt_to_chroma(F_po_o_est), isa.cqt_to_chroma(F_o))
    D, B, po_o_path = dtw.dtw(C, np.array([[1,1],[1,2],[2,1]]),[1,1,2], True)
    po_o_path[0,:] += po_offset # account for offset in PO-O alignment
    timestamps.append(time.time())
    
    # infer piano-orchestra alignment
    wp_AC = align_tools.infer_alignment(wp_AB, po_o_path, frames=True) # inferred P-O alignment
    timestamps.append(time.time())
    
    np.save(f'{out_dir}/hyp.npy', wp_AC*hop_sec)
    np.save(f'{out_dir}/p_po_align.npy', wp_AB)
    np.save(f'{out_dir}/runtime.npy', np.array(timestamps))
    
    return

In [ ]:
def verify_hyp_dir(indir):
    '''
    Verifies that the specified scenario hypothesis directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/hyp.npy'), f'{indir} is missing the required files, please re run the online processing'

# Example

Here is an example of how to call the offline and online processing functions on a scenario directory.

In [ ]:
# scenario_dir = 'scenarios/s2'
# out_dir = 'experiments/test/s2'
# cache_dir = 'experiments/test/cache'
# hop_size = 512
# steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# weights = np.array([2,3,3])
# offline_processing(scenario_dir, cache_dir, hop_size, steps, weights)
# online_processing(scenario_dir, out_dir, cache_dir, hop_size, steps, weights)